Test Phase4_3d file

In [1]:
# install anaconda: https://www.anaconda.com/products/individual
# if use Visual Studio Code, install python extension and select the python interpreter that includes anaconda
# install micropip: https://pypi.org/project/micropip/. You can also run anaconda prompt and type "pip install micropip"
import numpy as np
import micropip
await micropip.install("scipy")

In [2]:
# from Phase4_trace_3d import Phase4_trace_3d
from generateTestPositions import generateTestPositions
import generateTestPositions
from generateTestPositions import get_velocity_function
from generateTestPositions import generateTestPositions
from generateTestPositions import get_initial_position
# Cell 4
# Testing with dummy data
NumOfDataPoints = 1
clusterness = 0.1 # smaller number the more clustered
# Input conditions
initial_positions = []
v = []
# for i in range(1):
#     [x,y,z] = get_initial_position(i)
#     initial_positions.append([x, y, z])
#     vel = get_velocity_function(i)
#     v.append(vel)
initial_positions = np.zeros((NumOfDataPoints,3))
initial_positions[0] = get_initial_position(0)
# v.append(lambda t: [20*np.sin(t), 20*np.cos(t), 20])
v.append(lambda t: [0,0,0])
noise = 1e-3
theta_degrees = 1.8
rev = 2  # revolutions of camera for the entire process
NOS = int(rev * 360 / theta_degrees)
NOS = 54
NOS_per_section = 20  # must be larger than 5 to satisfy equations
camera_speed = 1/(0.015*200)  # in Hz or revolution per second
SRD = 20  # mm, Source-Reference Distance
RDD = 480  # mm, Reference-Detector (screen) Distance
radius = 1

pixelResolution = 0.172  # every pixel is equal to mm
method = 'acceleration'
dataPiling = 'overlap'

# Auto-calculations of the rest of the parameters derived from the setting above
delta_T = camera_speed * theta_degrees / 360
shots_per_second = 1 / delta_T

# Define the velocity function
# v = lambda t: [0.9 * np.sin(t), 0.9 * np.cos(t), 1]

# Pack conditions into a list
conditions = [noise, delta_T, NOS, theta_degrees, NOS_per_section, SRD, RDD,method,dataPiling]
print(conditions)

xz_proj = np.zeros((NOS, NumOfDataPoints*2))

real_positions = np.zeros((NOS, NumOfDataPoints*3))
# Generate test positions
for i in range(NumOfDataPoints):
    vel = v[i]
    xz_proj[:,i*2:i*2+2], real_positions[:,i*3:i*3+3]= generateTestPositions(vel, initial_positions[i], conditions)

print("xz_proj, ",xz_proj)
print(xz_proj.shape)

[0.001, 0.0016666666666666666, 54, 1.8, 20, 20, 480, 'acceleration', 'overlap']
xz_proj,  [[ 5.00069862e+00 -2.50729656e-04]
 [ 5.10515293e+00  1.56117726e-04]
 [ 5.25480090e+00 -6.57563473e-06]
 [ 5.40011949e+00  9.39524283e-04]
 [ 5.53921287e+00  5.22430016e-04]
 [ 5.67327032e+00  1.09230977e-03]
 [ 5.80292181e+00 -1.41518776e-04]
 [ 5.92561701e+00  7.82436085e-04]
 [ 6.04224264e+00  6.46552522e-04]
 [ 6.15480238e+00  5.54674072e-04]
 [ 6.25999931e+00  1.16671899e-04]
 [ 6.35970863e+00 -3.67128521e-04]
 [ 6.45406475e+00 -4.42878355e-04]
 [ 6.54040587e+00 -3.65305000e-05]
 [ 6.62075730e+00 -2.79623796e-04]
 [ 6.69597557e+00  5.02033715e-04]
 [ 6.76361135e+00 -1.90691622e-04]
 [ 6.82570437e+00  8.05631761e-04]
 [ 6.87975805e+00 -8.22869658e-04]
 [ 6.92657123e+00 -7.01333473e-04]
 [ 6.96852814e+00  8.49712819e-04]
 [ 7.00314375e+00 -4.11847750e-05]
 [ 7.02988618e+00 -3.03550896e-04]
 [ 7.05101969e+00 -4.47203282e-04]
 [ 7.06478231e+00  3.14777687e-04]
 [ 7.07097085e+00 -1.68731093e-04]


In [3]:
# Print the first few rows of the generated data
# print("xz_proj:")
# print(xz_proj[:])
# print("real_positions:")
# print(real_positions[:])

In [4]:
import pandas as pd
import os


def rename_files_replace_space(directory_path):
    # Get a list of all files in the directory
    filenames = os.listdir(directory_path)
    
    for filename in filenames:
        # Replace spaces with underscores
        new_filename = filename.replace('_', '')
        
        # Construct the full old and new file paths
        old_filepath = os.path.join(directory_path, filename)
        new_filepath = os.path.join(directory_path, new_filename)
        
        # Rename the file
        os.rename(old_filepath, new_filepath)

In [5]:
# Import the function directly from the module
from Phase4_trace_3d import Phase4_trace_3d
from smooth_points import smooth_points
from particle_path_finder import ParticlePathFinder as pf
# from particle_path_finder import OriginalPathKeeper as opk
import random
import numpy as np
import pandas as pd
import os


# # read data from .csv file
# input_data = pd.read_csv('particle_tracking_input.csv')

# columnIndex = 0

theta_degrees = 1.8
# # Ai recognition caller
alpha = np.radians(theta_degrees)  # Example rotation angle in radians
path_finder = pf(alpha)
# original_path_keeper = opk(alpha)
# iterate through each row of the input data as a snapshot, and put the snapshot into the particle path finder
# rename_files_replace_space("Data_3particles")

# print(os.listdir("Data_3particles"))
sorted_filenames = sorted(os.listdir("Data_3particles"), key=lambda x: int(x.split('Shot')[1].split('.csv')[0]))
print(sorted_filenames)
offset = [487/2,195/2]
for file in sorted_filenames:
    if file.endswith(".csv"):
        filename = os.path.join("Data_3particles", file)
        input_data = pd.read_csv(filename, header=None)
        input_data = np.array(np.transpose(input_data))
        values =  input_data[0]
        print("values:", values)
        print("read file: ", filename)

        paired_values = []
        i = 0
        for j in range(len(values)//2):
            print("values[i:i+2], ", values[i:i+2])
            print("offset: ", offset)
            input_values = values[i:i+2] - offset
            inputList = input_values*pixelResolution
           
            # input format, list of tuple of two elements (x,y)
            paired_values.append(inputList)

            # scambled_values = random.shuffle(paired_values.copy())
            # print("paired values:", paired_values)
            
            i += 2

        print("paired values:", paired_values)
        path_finder.append(paired_values)
    
    

    

# estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
# # Then you can call it just by its name:
# for i in range(NumOfDataPoints):
#     estimated_positions_single = Phase4_trace_3d(conditions, xz_proj[:,i*2:i*2+2])
#     estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

#     estimated_positions[:,i*3:i*3+3] = estimated_positions_single
sorted_particle_data = path_finder.get_particle_data()
print("sorted_positions: ", sorted_particle_data[3])



['Shot0.csv', 'Shot1.csv', 'Shot2.csv', 'Shot3.csv', 'Shot4.csv', 'Shot5.csv', 'Shot6.csv', 'Shot7.csv', 'Shot8.csv', 'Shot9.csv', 'Shot10.csv', 'Shot11.csv', 'Shot12.csv', 'Shot13.csv', 'Shot14.csv', 'Shot15.csv', 'Shot16.csv', 'Shot17.csv', 'Shot18.csv', 'Shot19.csv', 'Shot20.csv', 'Shot21.csv', 'Shot22.csv', 'Shot23.csv', 'Shot24.csv', 'Shot25.csv', 'Shot26.csv', 'Shot27.csv', 'Shot28.csv', 'Shot29.csv', 'Shot30.csv', 'Shot31.csv', 'Shot32.csv', 'Shot33.csv', 'Shot34.csv', 'Shot35.csv', 'Shot36.csv', 'Shot37.csv', 'Shot38.csv', 'Shot39.csv', 'Shot40.csv', 'Shot41.csv', 'Shot42.csv', 'Shot43.csv', 'Shot44.csv', 'Shot45.csv', 'Shot46.csv', 'Shot47.csv', 'Shot48.csv', 'Shot49.csv', 'Shot50.csv', 'Shot51.csv', 'Shot52.csv', 'Shot53.csv']
values: [243.5  97.5 294.5  77.5 183.5 111.5]
read file:  Data_3particles\Shot0.csv
values[i:i+2],  [243.5  97.5]
offset:  [243.5, 97.5]
values[i:i+2],  [294.5  77.5]
offset:  [243.5, 97.5]
values[i:i+2],  [183.5 111.5]
offset:  [243.5, 97.5]
paired val

In [6]:
# print("sorted_positions type: ", type(sorted_particle_data[0]['coords']))
# print("sorted_positions: ", np.array(sorted_particle_data[0]['coords']))

# for i in range(NumOfDataPoints):
#     estimated_positions_single = Phase4_trace_3d(conditions, xz_proj[:,i*2:i*2+2])
#     estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

#     estimated_positions[:,i*3:i*3+3] = estimated_positions_single
NumOfDataPoints = len(sorted_particle_data)
print("NumOfDataPoints: ", NumOfDataPoints)
estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
for i in range(NumOfDataPoints):
    estimated_positions_single = Phase4_trace_3d(conditions, np.array(sorted_particle_data[i]['coords']))
    estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

    estimated_positions[:,i*3:i*3+3] = estimated_positions_single
# estimated_positions_single = Phase4_trace_3d(conditions, np.array(sorted_particle_data[0]['coords']))

# estimated_positions = estimated_positions_single
estimated_positions_single = Phase4_trace_3d(conditions, xz_proj)
estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

estimated_positions =np.concatenate((estimated_positions,estimated_positions_single),axis=1)
print('estimated positions: \n',estimated_positions)
# print('real positions: \n',real_positions)

NumOfDataPoints:  4
NOS:  54
NOS_per_Section:  20
position_rotated -20.104623373483417


position_rotated -20.175929367789617
position_rotated -20.222695488966785
position_rotated -20.231748440137125
position_rotated -20.195512383380336
position_rotated -20.11191869206273
position_rotated -19.98394140608327
position_rotated -19.81947866164427
position_rotated -19.63116461103922
position_rotated -19.435380116153365
position_rotated -19.353423689370896
position_rotated -19.03370157776121
position_rotated -19.01622658028132
position_rotated -18.351282819958975
position_rotated -18.302513028033527
position_rotated -18.26472116077838
position_rotated -18.14190580545656
position_rotated -17.900227304993194
position_rotated -17.557325111170385
position_rotated -17.13119505900797
position_rotated -16.661752174855376
position_rotated -16.203609631689005
position_rotated -15.785466132038506
position_rotated -15.412764528927449
position_rotated -15.090930895931427
position_rotated -14.789314317020668
position_rotated -14.376533809650061
position_rotated -13.720761012240596
position_r

In [7]:
import matplotlib
print(matplotlib.get_backend())
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')  # or another interactive backend
from Plotting import plotting_single
# from mpl_toolkits.mplot3d import Axes3D
import os

# Create the figure and axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(NumOfDataPoints+1):
    plotting_single(estimated_positions[:,i*3:i*3+3],i,ax)
# def Phase4Graph(real_positions, estimated_positions, conditions, v, f2, NumOfDataPoints):
#     noise, delta_T, NOS, theta_degree, N, SRD, RDD, method, dataPiling = conditions
    
#     plt.ion()
#     # Ensure that real_positions and estimated_positions are the same length
#     real_positions = real_positions[:len(estimated_positions)]
    
#     fig = plt.figure(f2, figsize=(20, 16), dpi=100)
    
#     ax1 = fig.add_subplot(111, projection='3d')
    
#     for i in range(NumOfDataPoints):
#         # ax1.plot(real_positions[:, i*3], real_positions[:, i*3+1], real_positions[:, i*3+2], 'r', markersize=10)
#         ax1.plot(estimated_positions[:, i*3], estimated_positions[:, i*3+1], estimated_positions[:, i*3+2], 'b', markersize=20)
    
#     ax1.set_xlabel('X')
#     ax1.set_ylabel('Y')
#     ax1.set_zlabel('Z')
#     # ax1.set_xlim([-0.5, 0.5])
#     # ax1.set_ylim([-0.5, 0.5])
#     ax1.set_zlim([0, 1.5])
#     ax1.set_title('Multi Particle Graphing 3D')
#     ax1.legend(['Particle 1', 'Particle 2', 'Particle 3', 'Particle 4'])
    
    # distances = np.linalg.norm(real_positions - estimated_positions, axis=1)
    # min_val, min_idx = np.min(distances), np.argmin(distances)
    # max_val, max_idx = np.max(distances), np.argmax(distances)
    
    # ax1.text(*real_positions[min_idx], f'Min ({min_idx}): {real_positions[min_idx]}', color='r')
    # ax1.text(*estimated_positions[min_idx], f'Min ({min_idx}): {estimated_positions[min_idx]}', color='b')
    # ax1.text(*real_positions[max_idx], f'Max ({max_idx}): {real_positions[max_idx]}', color='r')
    # ax1.text(*estimated_positions[max_idx], f'Max ({max_idx}): {estimated_positions[max_idx]}', color='b')
    
    # print(f'Noise: {noise}\nMin Distance: {min_val}\nMax Distance: {max_val}\nNumber of Shots: {NOS}\nRotation: {theta_degree}\nTime Between Shots: {delta_T}\nNOS Per Section: {N}\nMethod: {method}\nData Piling: {dataPiling}')
    
    # v_str = str(v).replace('<function ', '').replace(' at 0x' + hex(id(v))[2:] + '>', '')
    
    # print(f'True velocity:\n{v_str}\nt is time elapsed')
    
    
plt.show()


module://matplotlib_inline.backend_inline


In [ ]:
# f2 = 0
# Phase4Graph(real_positions, estimated_positions, conditions, v, f2, len(sorted_particle_data))